# Using EcoFOCIpy to process raw field data

## DY2208L2 (BASIS Leg II - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [21]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [22]:
sample_data_dir = '/Users/bell/ecoraid/2022/CTDcasts/dy2208l2/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [23]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY2208L2' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY2208L2.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml'
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd017.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd019.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd020.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd021.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd022.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd023.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd024.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd025.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd026.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd027.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd028.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd029.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconvert

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd031.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd032.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd033.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd034.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd035.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2208l2/rawconverted/ctd036.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

In [24]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd017.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sal00,sal11,sigma-t00,sigma-t11,sbeox1ml/l,sbeox1ps,sbox1mm/kg,fleco-afl,t090c,t190c,prdm,turbwetntu0,par,scan,datetime
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8
mean,6.415138,95.310750,279.703125,31.272900,31.271650,24.322750,24.321687,6.138738,91.211000,267.652750,1.518500,8.082450,8.083187,19.927875,1.582050,0.641070,10926.500000,2022-09-02 06:42:40.750000128
min,5.775000,81.423000,251.685000,31.183500,31.182600,24.081100,24.080500,5.508300,77.677000,240.063000,0.351500,5.891600,5.894400,3.376000,0.451700,0.283650,5583.000000,2022-09-02 06:38:58
25%,5.827225,82.786250,253.977000,31.184400,31.183250,24.081700,24.080900,5.564375,79.049250,242.521250,0.541425,6.224450,6.223500,8.550500,1.468425,0.283650,8622.500000,2022-09-02 06:41:04.750000128
50%,6.774150,103.062500,295.413500,31.205950,31.204200,24.105450,24.103650,6.493450,98.786500,283.171500,2.074600,9.280600,9.283500,15.182500,1.846700,0.283670,11914.500000,2022-09-02 06:43:22
75%,6.791500,103.398750,296.173750,31.386375,31.385450,24.671050,24.670425,6.506050,99.062500,283.729000,2.170275,9.324725,9.324325,31.701750,1.859625,0.605602,13370.750000,2022-09-02 06:44:22.249999872
max,6.792100,103.418000,296.204000,31.423300,31.421600,24.740200,24.738600,6.508800,99.103000,283.849000,2.224300,9.325100,9.325200,43.013000,1.863700,1.855200,14457.000000,2022-09-02 06:45:08
std,0.510534,10.999125,22.343475,0.112320,0.112363,0.321358,0.321421,0.501785,10.717928,21.958344,0.867244,1.695857,1.696184,15.795090,0.489859,0.666127,3360.897287,NaN


## Create BTL report file

In [25]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [26]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY2208L2',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': 'L2',
 'Vessel': 'NOAAS Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2022, 9, 2),
 'EndDate': datetime.date(2022, 9, 9),
 'Project': 'BASIS',
 'ChiefScientist': 'Adam Spear',
 'StartPort': 'Dutch Harbor, AK',
 'EndPort': 'Kodiak, AK',
 'CruiseLocation': 'Bering Sea',
 'Description': 'BASIS',
 'CruiseYear': 2022,
 'ctdlogs_pdf_name': 'DY22-08_COD_SHEETS.pdf'}

In [27]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD017']

{'id': 54074,
 'Vessel': 'NOAAS Oscar Dyson',
 'CruiseID': 'DY2208',
 'Project_Leg': 'L2',
 'UniqueCruiseID': 'DY2208L2',
 'Project': 'BASIS',
 'StationNo_altname': 's18h2',
 'ConsecutiveCastNo': 'CTD017',
 'LatitudeDeg': 57,
 'LatitudeMin': 59.688,
 'LongitudeDeg': 165,
 'LongitudeMin': 0.0082,
 'GMTDay': 2,
 'GMTMonth': 'Sep',
 'GMTYear': 2022,
 'GMTTime': 52320,
 'DryBulb': -99.9,
 'RelativeHumidity': 95,
 'WetBulb': -99.9,
 'Pressure': -10000,
 'SeaState': '-999',
 'Visibility': '-99',
 'WindDir': -1000,
 'WindSpd': -99.0,
 'CloudAmt': '-99',
 'CloudType': '-99',
 'Weather': '-99',
 'SurfaceTemp': -99.9,
 'BottomDepth': 48,
 'StationNameID': '',
 'MaxDepth': 42,
 'InstrumentSerialNos': '',
 'Notes': '',
 'NutrientBtlNiskinNo': '1;2;3;4;5;6',
 'NutrientBtlNumbers': '116;117;118;119;120;121\r',
 'OxygenBtlNiskinNo': '',
 'OxygenBtlNumbers': '',
 'SalinityBtlNiskinNo': '',
 'SalinityBtlNumbers': '',
 'ChlorophyllBtlNiskinNo': '',
 'ChlorophyllBtlVolumes': '',
 'InstrumentType': 'SBE91

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [28]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [29]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd017.btl'] = cruise_data['ctd017.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'pressure',
                        'flag':'flag'})

cruise_data['ctd017.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,oxy_concM_ch2,oxy_percentsat_ch2,oxy_conc_ch2,chlor_fluorescence,temperature_ch1,temperature_ch2,pressure,turbidity,par,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,,
8.0,6.7915,103.396,296.173,31.1905,31.1876,24.0876,24.085,6.5019,98.987,283.545,2.1663,9.3179,9.3201,10.326,1.8606,0.28367,13055.0,2022-09-02 06:44:09,ctd017


In [30]:
cruise_data['ctd017.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'salinity_ch1',
       'salinity_ch2', 'sigma_t_ch1', 'sigma_t_ch2', 'oxy_concM_ch2',
       'oxy_percentsat_ch2', 'oxy_conc_ch2', 'chlor_fluorescence',
       'temperature_ch1', 'temperature_ch2', 'pressure', 'turbidity', 'par',
       'scan', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [31]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [32]:
cruise_data['ctd017.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'salinity_ch1',
       'salinity_ch2', 'sigma_t_ch1', 'sigma_t_ch2', 'oxy_concM_ch2',
       'oxy_percentsat_ch2', 'oxy_conc_ch2', 'chlor_fluorescence',
       'temperature_ch1', 'temperature_ch2', 'pressure', 'turbidity', 'par',
       'scan', 'datetime', 'cast'],
      dtype='object')

In [33]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd017.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [34]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8)
Coordinates:
  * bottle              (bottle) float64 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0
Data variables: (12/19)
    oxy_concM_ch1       (bottle) float64 5.775 5.78 5.843 ... 6.792 6.789 6.792
    oxy_percentsat_ch1  (bottle) float64 81.42 81.5 83.21 ... 103.4 103.4 103.4
    oxy_conc_ch1        (bottle) float64 251.7 251.9 254.7 ... 296.2 296.1 296.2
    salinity_ch1        (bottle) float64 31.42 31.42 31.37 ... 31.19 31.18 31.18
    salinity_ch2        (bottle) float64 31.42 31.42 31.37 ... 31.19 31.18 31.18
    sigma_t_ch1         (bottle) float64 24.74 24.74 24.65 ... 24.09 24.08 24.08
    ...                  ...
    pressure            (bottle) float64 43.01 39.99 28.94 ... 10.33 3.376 3.509
    turbidity           (bottle) float64 1.852 1.473 0.4517 ... 1.841 1.859
    par                 (bottle) float64 0.2837 0.2837 0.2837 ... 1.855 1.571
    scan                (bottle) float64 5.583e+03 7.01e+03 ... 1.446e+04
    datetime            (bottle) datetime64[ns] 2022-09-02T06:38:58 ... 2022-...
    cast                (bottle) object 'ctd017' 'ctd017' ... 'ctd017' 'ctd017'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [35]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [36]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd017.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [37]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/19)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 5.775 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 81.42 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 251.7 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 31.42 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 31.42 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 24.74 ......
    ...                  ...
    pressure            (latitude, longitude, time, bottle) float64 43.01 ......
    turbidity           (latitude, longitude, time, bottle) float64 1.852 ......
    par                 (latitude, longitude, time, bottle) float64 0.2837 .....
    scan                (latitude, longitude, time, bottle) float64 5.583e+03...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd017' ....

In [38]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD017')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/19)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 5.775 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 81.42 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 251.7 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 31.42 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 31.42 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 24.74 ......
    ...                  ...
    pressure            (latitude, longitude, time, bottle) float64 43.01 ......
    turbidity           (latitude, longitude, time, bottle) float64 1.852 ......
    par                 (latitude, longitude, time, bottle) float64 0.2837 .....
    scan                (latitude, longitude, time, bottle) float64 5.583e+03...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd017' ....
Attributes: (12/44)
    CruiseID:                      DY2208L2
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   L2
    Vessel:                        NOAAS Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2024-06-25T15:23:02Z
    date_modified:                 
    QC_indicator:                  unqcd

## Rare Bottle File Edits

<div class="warning" style='background-color:#ffcccb; color: #FF0000; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>WARNING</b></p>
<p style='margin-left:1em;'>bottle/niskin and rosette position should be the same but can be different (example, bottles are labeled sequentially but a rosette position is skipped due to balancing or other instruments.  On this cruise the following rosette positin was fired, while the bottles where labled differently</p>
cruise_data[cast]
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0lax"><span style="font-weight:bold">Rosette</span></th>
    <th class="tg-0lax">3</th>
    <th class="tg-0lax">4</th>
    <th class="tg-0lax">5</th>
    <th class="tg-0lax">6</th>
    <th class="tg-0lax">7<br></th>
    <th class="tg-0lax">8</th>
    <th class="tg-0lax">9</th>
    <th class="tg-0lax">10</th>
    <th class="tg-0lax">11</th>
    <th class="tg-0lax">12</th>
    <th class="tg-0lax">1</th>
    <th class="tg-0lax">2</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax"><span style="font-weight:bold">Niskin</span><br></td>
    <td class="tg-0lax">1</td>
    <td class="tg-0lax">2</td>
    <td class="tg-0lax">3</td>
    <td class="tg-0lax">4</td>
    <td class="tg-0lax">5</td>
    <td class="tg-0lax">6</td>
    <td class="tg-0lax">7</td>
    <td class="tg-0lax">8</td>
    <td class="tg-0lax">9</td>
    <td class="tg-0lax">10<br></td>
    <td class="tg-0lax">11</td>
    <td class="tg-0lax">12</td>
  </tr>
</tbody>
</table>
</div>

In [39]:
import pandas as pd
nisk_btl = pd.DataFrame(data=[[3,1],[4,2],[5,3],[6,4],[7,5],[8,6],[9,7],[10,8],[11,9],[12,10],[1,11],[2,12]],columns=['rosette','bottle']) 
for cast in cruise_data.keys():
    cruise_data[cast] = pd.merge(nisk_btl, cruise_data[cast], left_on='rosette', right_on='bottle').drop('rosette',axis=1).set_index('bottle')      

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [40]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncC

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history